In [2]:
import torch
import torch.nn as nn
from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer, LlamaForCausalLM

import sys, os, json, math
from tqdm import tqdm
import numpy as np


std = 0.012528747320175171

/home/jgryu/miniconda3/envs/nic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### AWQ

In [ ]:
def latest_version_path(cache_dir, model_name, branch = 'main'):
    model_name_dir =  "models--" + model_name.replace('/', '--')
    path = os.path.join(cache_dir, model_name_dir)
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

cache_directory = "../Wparam_dataset_v0/model_zoo/huggingface" 
ckpt_path = latest_version_path(cache_directory, 'meta-llama/Meta-Llama-3-8B')
net = LlamaForCausalLM.from_pretrained(ckpt_path, local_files_only=True)

# ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
# net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
state_dict = net.state_dict()


In [ ]:
bpps = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12]
# bpps = [4]
mses = []
mse_fn = nn.MSELoss()
for bpp in bpps:
    ckpt_path = f'../model_cache_reconstructed/awq/llama3-8b-my-w{bpp}-g128-fake-quantized'
    recon_net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
    recon_state_dict = recon_net.state_dict()

    n = 0
    mse = 0
    mse_layer = []
    for k, v in state_dict.items():
        if 'mlp' not in k and 'self_attn' not in k: continue
        
        mse += ((recon_state_dict[k] - v)**2).sum()
        n += v.numel()
        print(k, mse_fn(recon_state_dict[k], v)/std**2)
        mse_layer.append(mse_fn(recon_state_dict[k], v)/std**2)
    mse = mse / n / std **2 
    mses.append(mse.item())
    print(mse)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(mse_layer, label='awq')
plt.plot(mse_, label='vqvae')
# plt.plot(mse_2, label='vqvae_idx')
plt.legend()
plt.yscale('log')
plt.show()

### AWQ
bpp:  [2, 3, 4, 5, 6, 7, 8, 9, 10, 12]

mse:  [23.015626907348633, 21.956884384155273, 18.508888244628906, 189.52224731445312, 21.462759017944336, 18.203344345092773, 23.026723861694336, 12.113859176635742, 13.774173736572266, 57.454349517822266]

In [ ]:
print('bpp: ', bpps)
print('mse: ', mses)

### VQ

In [ ]:
def latest_version_path(cache_dir, model_name, branch = 'main'):
    model_name_dir =  "models--" + model_name.replace('/', '--')
    path = os.path.join(cache_dir, model_name_dir)
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

cache_directory = "../Wparam_dataset_v0/model_zoo/huggingface" 
ckpt_path = latest_version_path(cache_directory, 'meta-llama/Meta-Llama-3-8B')
net = LlamaForCausalLM.from_pretrained(ckpt_path, local_files_only=True)

# ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
# net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
state_dict = net.state_dict()


In [ ]:
root_dir = '/home/jgryu/Weight_compression/model_cache_reconstructed/vqvae_idx_v1/per_row_16_calib'
root_dir = '../model_cache_reconstructed/vqvae_corrected_scale/mlp_attn_16_row_dataset.pt'
root_dir = '../model_cache_reconstructed/vqvae_idx_v2/per_row_16_calib'
root_dir = '/home/jgryu/Weight_compression/model_cache_reconstructed/vqvae_idx_v2_random/per_row_16_calib'
root_dir = '/home/jgryu/Weight_compression/model_cache_reconstructed/vqvae_idx_v2_random_all/per_row_16_calib'
import glob
ckpt_paths = glob.glob(os.path.join(root_dir, "**/*.pth.tar"), recursive=True)
ckpt_path_list = []

for ck in ckpt_paths:
    # if 'nmse' in ck: continue////// 
    ckpt_path_list.append(ck)

ckpt_path_list = ['/home/jgryu/Weight_compression/model_cache_reconstructed/vqvae_calib__/per_row_16_calib/bpp4.0_size16_nmse_neNone_de16_K8_P8_encdim512_batch_size2048_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.01304_total_iter_1100000.pth.tar']
ckpt_path_list = ['/home/jgryu/Weight_compression/model_cache_reconstructed/vqvae_idx_v2/per_row_16_calib/bpp4.0_size16_smse_neNone_de16_K8_P8_encdim512_batch_size2048_total_iter1500000_lr0.0001_seed100/best_mse_model_MSE_0.00994_total_iter_900000.pth.tar']
mses = []
mse_fn = nn.MSELoss()
for ckpt_path in ckpt_path_list:
    recon_net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
    recon_state_dict = recon_net.state_dict()

    n = 0
    mse = 0
    mse_2 = []
    for k, v in state_dict.items():
        if 'mlp' not in k and 'attn' not in k: continue
        # if 'mlp' not in k: continue
        
        
        mse += ((recon_state_dict[k] - v)**2).sum()
        n += v.numel()
        mse_2.append(mse_fn(recon_state_dict[k], v)/std**2)
    mse = mse / n / std **2 
    mses.append(mse.item())
    print(ckpt_path.split('/')[-2])
    print(mse)

# SeedLM

In [ ]:
def latest_version_path(cache_dir, model_name, branch = 'main'):
    model_name_dir =  "models--" + model_name.replace('/', '--')
    path = os.path.join(cache_dir, model_name_dir)
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

cache_directory = "../Wparam_dataset_v0/model_zoo/huggingface" 
ckpt_path = latest_version_path(cache_directory, 'meta-llama/Meta-Llama-3-8B')
net = LlamaForCausalLM.from_pretrained(ckpt_path, local_files_only=True)

# ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
# net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
state_dict = net.state_dict()

ckpt_path = f'../model_cache_reconstructed/seedlm/bpp4.0_C8_P3_K16'
recon_net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
recon_state_dict = recon_net.state_dict()

n = 0
mse = 0
for k, v in state_dict.items():
    if 'mlp' not in k and 'attn' not in k: continue
    
    mse += ((recon_state_dict[k] - v)**2).sum()
    n += v.numel()
    
mse = mse / n / std **2 
print(mse)

In [ ]:
print(mse)

# RTN

In [3]:
def latest_version_path(cache_dir, model_name, branch = 'main'):
    model_name_dir =  "models--" + model_name.replace('/', '--')
    path = os.path.join(cache_dir, model_name_dir)
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

cache_directory = "../Wparam_dataset_v0/model_zoo/huggingface" 
ckpt_path = latest_version_path(cache_directory, 'meta-llama/Meta-Llama-3-8B')
net = LlamaForCausalLM.from_pretrained(ckpt_path, local_files_only=True)

# ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
# net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
state_dict = net.state_dict()

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.02s/it]


In [4]:
root_dir = '/home/jgryu/Weight_compression/model_reconstructed/rtn'
import glob
ckpt_paths = glob.glob(os.path.join(root_dir, "**/*"), recursive=True)
ckpt_path_list = []

for ck in ckpt_paths:
    if 'result' in ck: continue
    ckpt_path_list.append(ck)

mses = []
mse_fn = nn.MSELoss()
for ckpt_path in ckpt_path_list:
    try:
        recon_net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
        recon_state_dict = recon_net.state_dict()

        n = 0
        mse = 0
        mse_2 = []
        for k, v in state_dict.items():
            if 'mlp' not in k and 'attn' not in k: continue
            # if 'mlp' not in k: continue
            
            
            mse += ((recon_state_dict[k] - v)**2).sum()
            n += v.numel()
            mse_2.append(mse_fn(recon_state_dict[k], v)/std**2)
        mse = mse / n / std **2 
        mses.append(mse.item())
        print(ckpt_path.split('/')[-1])
        print(mse)
    except:
        pass

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.75s/it]


b5_g128
tensor(0.0025)


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.83s/it]


b7_g-1
tensor(0.0004)


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.76s/it]


b5_g-1
tensor(0.0060)


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.39s/it]


b7_g128
tensor(0.0002)


In [ ]:
bpps = [3, 4]
mses = []
mse_fn = nn.MSELoss()
for bpp in bpps:
    ckpt_path = f'../model_reconstructed/rtn/bpp{bpp}'
    recon_net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
    recon_state_dict = recon_net.state_dict()

    n = 0
    mse = 0
    mse_layer = []
    for k, v in state_dict.items():
        # if 'mlp' not in k and 'attn' not in k: continue
        
        mse += ((recon_state_dict[k] - v)**2).sum()
        n += v.numel()
        print(k, mse_fn(recon_state_dict[k], v)/std**2)
        mse_layer.append(mse_fn(recon_state_dict[k], v)/std**2)
    mse = mse / n / std **2 
    mses.append(mse.item())
    print(mse)

In [ ]:
ckpt_path_list